In [28]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from utils.utils_chain import WrapperAddress
from multiversx_sdk.abi import Abi

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"

GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)
DOCKER_URL = PROJECT_ROOT / "docker"

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api
context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)

wasm_path = "/home/multiversx/Documents/DEX/git-main/mx-sdk-py-exchange/wasm/mex-governance/mex-governance.wasm"
contract_code_hash = "ae5c0171d34f2bac384b6d38184f89abd7ef05a4d5d9b973db3c58015bdf587f"

2025-06-02 11:10:40,121 - WARNING - Structure definition for mex_governance not found in /home/multiversx/Documents/DEX/git-main/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json


Shadowfork detected. Owner: erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97
loaded 1 accounts from 1 PEM files.


2025-06-02 11:10:40,989 - DEBUG - Account.sync_nonce() done: 5671
2025-06-02 11:10:41,222 - DEBUG - Account.sync_nonce() done: 5671
2025-06-02 11:10:41,223 - WARNING - Loaded 16 tokens instead of expected 15.
2025-06-02 11:10:41,445 - DEBUG - Account.sync_nonce() done: 5671
2025-06-02 11:10:41,447 - INFO - Loaded 3 egld_wraps out of expected 3.
2025-06-02 11:10:41,449 - INFO - Loaded 1 locked_assets out of expected 1.
2025-06-02 11:10:41,450 - INFO - Loaded 1 proxies out of expected 1.
2025-06-02 11:10:41,451 - INFO - Loaded 0 simple_locks out of expected 0.
2025-06-02 11:10:41,452 - INFO - Loaded 1 simple_locks_energy out of expected 1.
2025-06-02 11:10:41,453 - INFO - Loaded 1 fees_collectors out of expected 1.
2025-06-02 11:10:41,453 - INFO - Loaded 1 unstakers out of expected 1.
2025-06-02 11:10:41,454 - INFO - Loaded 1 proxies_v2 out of expected 1.
2025-06-02 11:10:41,454 - INFO - Loaded 1 position_creator out of expected 1.
2025-06-02 11:10:41,455 - INFO - Loaded 1 locked_token_p

In [2]:
import json
import subprocess
from time import sleep
from typing import Any

from utils.utils_chain import Account


def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    
def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)

# @pytest.fixture
def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})

def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

In [3]:
from utils.utils_chain import Account, WrapperAddress as Address


USERS = [
        "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw", 
        "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm",
        "erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n",
        "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97" # DEX OWNER
]

BASE_TOKEN = "MEX-455c57"


users = users_init()
user_account = Account(users[2].address.to_bech32())
user_account.sync_nonce(context.network_provider.proxy)
user_account.signer = context.deployer_account.signer

2025-05-30 09:35:50,307 - DEBUG - Account.sync_nonce() done: 4422
2025-05-30 09:35:50,312 - DEBUG - Account.sync_nonce() done: 2729
2025-05-30 09:35:50,320 - DEBUG - Account.sync_nonce() done: 0
2025-05-30 09:35:50,325 - DEBUG - Account.sync_nonce() done: 0
2025-05-30 09:35:50,331 - DEBUG - Account.sync_nonce() done: 4422
2025-05-30 09:35:50,335 - DEBUG - Account.sync_nonce() done: 0


erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97


In [4]:
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from contracts.router_contract import RouterContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.mex_governance_contract import MEXGovernanceContract
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

# mex_governance_contract: MEXGovernanceContract
# mex_governance_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

energy_contract: SimpleLockEnergyContract
router_contract: RouterContract
pair_contract: PairContract

pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgq0lzzvt2faev4upyf586tg38s84d7zsaj2jpsglugga")    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract = context.deploy_structure.get_deployed_contract_by_index(config.ROUTER_V2, 0)

# print(mex_governance_contract.address)
print(router_contract.address)
print(pair_contract.address)
print(mex_contract.address)


erd1qqqqqqqqqqqqqpgqq66xk9gfr4esuhem3jru86wg5hvp33a62jps2fy57p
erd1qqqqqqqqqqqqqpgq0lzzvt2faev4upyf586tg38s84d7zsaj2jpsglugga
erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2


UPGRADE ENERGY FACTORY

In [7]:
SIMPLE_LOCK_ENERGY_BYTECODE_PATH = "/home/multiversx/Documents/DEX/git-main/mx-sdk-py-exchange/wasm/energy-factory/energy-factory.wasm"
energy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

2025-05-30 09:37:17,298 - INFO - Upgrade simple lock energy contract
2025-05-30 09:37:17,301 - DEBUG - Upgrade SimpleLockEnergyContract contract
2025-05-30 09:37:17,306 - DEBUG - Upgrade arguments: []
2025-05-30 09:37:17,307 - DEBUG - Bytecode codehash: 4a79a41e418041ab23e5cd124a16869f4a4a9ceb348af51d4a6b2252b0cc294b
2025-05-30 09:37:17,317 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/0212b9f04d257ba71436cd7784031ec6a57c777dd50e46eb3afe2c82590c6316


'0212b9f04d257ba71436cd7784031ec6a57c777dd50e46eb3afe2c82590c6316'

In [8]:
advance_blocks(5)

In [ ]:
# erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp   UTK
from contracts.farm_contract import FarmContract


# farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 0)
# farms = []

farms = context.deploy_structure.get_deployed_contracts(config.FARMS_V2)
for farm in farms:
    print(farm.address)

DEPLOY FARMS

In [ ]:
from contracts.contract_identities import FarmContractVersion
from contracts.farm_contract import FarmContract

FARM_TOKEN_NAME = ""
FARM_TOKEN_TICKER = ""
POOL_ADDRESS = "erd1qqqqqqqqqqqqqpgqv0c22xku4vd5ugfh0u53tqufmdkqx5nv2jps8vsccx" #MEX-USDC
LOCK_EPOCHS = 1440
BOOSTED_YIELD_REWARDS = 6000
BOOSTED_YIELD_FACTORS = [2, 1, 0, 1, 1]
REWARDS_PER_BLOCK = 1
PENALTY_PERCENT = 300
MINIMUM_FARMING_EPOCH = 7

NO_OF_FARMS = 26
farm_contract: FarmContract
farm_contract = FarmContract(pair_contract.lpToken, "", energy_contract.base_token, "", FarmContractVersion.V2Boosted)

farms = []
admin_address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")

for i in range(NO_OF_FARMS):

    hash, address = farm_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                              config.FARM_V3_BYTECODE_PATH,
                                              [POOL_ADDRESS, context.admin_account.address.to_bech32(),])
    farm_contract.address = address
    farms.append(address)
    print(f"Farm contract deployed at {address}")

In [20]:
advance_blocks(5)

In [ ]:
for farm in farms:
    energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, farm)
    print(farm)

advance_blocks(10)

CONTRACT DEPLOY

In [6]:

reference_emission_rate = 10000
incentive_token = "MEX-455c57"
print(f"Energy contract address: {energy_contract.address}")
mex_governance_contract: MEXGovernanceContract = MEXGovernanceContract(reference_emission_rate,incentive_token,"")
print(mex_governance_contract.print_contract_info())
args = [
    reference_emission_rate,
    incentive_token, 
    energy_contract.address
    ]

hash, mex_governance_address = mex_governance_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, wasm_path, args)
print(f"Mex governance contract deployed at {mex_governance_address}")
mex_governance_contract.address = mex_governance_address

2025-05-30 09:36:11,808 - INFO - Deployed mex governance contract: 
2025-05-30 09:36:11,809 - INFO - Deploy MEXGovernanceContract contract
2025-05-30 09:36:11,810 - DEBUG - Deploy MEXGovernanceContract
2025-05-30 09:36:11,814 - DEBUG - Deploy arguments: [10000, 'MEX-455c57', erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3]
2025-05-30 09:36:11,815 - DEBUG - Bytecode codehash: 5e2e6f7508cab7ab671c1458d7b25109c5e1c81e9b9e0d665b5c7505ab9ea630
2025-05-30 09:36:11,823 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/cacb82147a45b63d45b1fd4545ce8668b4fe4860c5bfc49e01bb25bf08edd6d6


Energy contract address: erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3
None
Mex governance contract deployed at erd1qqqqqqqqqqqqqpgqhpjpk3tr93fsa76csgxdjsfj5qu8gaew2jpsplv2rr


In [6]:
from multiversx_sdk import Address

ex_governance_contract: MEXGovernanceContract = MEXGovernanceContract(reference_emission_rate,incentive_token,"")
mex_governance_contract.address = "erd1qqqqqqqqqqqqqpgq3nr7q49ez46sdmkt8pqj0qwk4k6wemu42jpsr7ayfm"

In [29]:
advance_blocks(5)

CONTRACT UPGRADE

In [ ]:
args = [
    reference_emission_rate,
    incentive_token, 
    energy_contract.address
    ]

mex_governance_contract.contract_upgrade(context.deployer_account,proxy,wasm_path, args, True)

In [24]:
address = "erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp"
context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
context.deployer_account.sync_nonce(proxy)
# Add the address to the unlocked token mint whitelist
energy_contract.add_to_unlocked_token_mint_whitelist(context.deployer_account, context.network_provider.proxy, [address])

2025-05-28 09:07:06,052 - DEBUG - Account.sync_nonce() done: 4421
2025-05-28 09:07:06,055 - INFO - Add to unlocked token mint whitelist
2025-05-28 09:07:06,058 - DEBUG - Calling addToUnlockedTokenMintWhitelist at erd1qqqqqqqqqqqqqpgq0tajepcazernwt74820t8ef7t28vjfgukp2sw239f3
2025-05-28 09:07:06,059 - DEBUG - Args: [erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp]
2025-05-28 09:07:06,065 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp]
2025-05-28 09:07:06,070 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/5d8d6d94a7ca175cdb08a3c7670fd83a7c819831407f9ae819722f4983c5902b


'5d8d6d94a7ca175cdb08a3c7670fd83a7c819831407f9ae819722f4983c5902b'

VOTE

In [25]:
# from multiversx_sdk import Address
# from utils import WrapperAddress

args: list[list[Any]] = []

abi = Abi.load(PROJECT_ROOT / "wasm/mex-governance/mex-governance.abi.json")

# farms = []
# farms.append([WrapperAddress("erd1qqqqqqqqqqqqqpgqshutpvdms2t92sychk5f0tkp5t4n4aap2jpsc0m7m3"), 0])

args.append((WrapperAddress("erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp"), 87908537530726782098814095450))
# args.append((Address()"erd1qqqqqqqqqqqqqpgqgewhrse0fzm5atnqd4epa49p58v3phuk2jpsthca8n", 0))

# args.append((WrapperAddress(farms[1]), 0))
# args.append((WrapperAddress(farms[2]), 0))
context.deployer_account.address = users[1].address
context.deployer_account.sync_nonce(proxy)

sc_args = [
    args
]
mex_governance_contract.vote(context.deployer_account, proxy, abi,sc_args)

2025-06-01 16:54:00,711 - DEBUG - Account.sync_nonce() done: 1027
2025-06-01 16:54:00,713 - INFO - Vote proposal
2025-06-01 16:54:00,715 - DEBUG - Calling vote at erd1qqqqqqqqqqqqqpgqhpjpk3tr93fsa76csgxdjsfj5qu8gaew2jpsplv2rr
2025-06-01 16:54:00,717 - DEBUG - Args: [[(erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp, 87908537530726782098814095450)]]
2025-06-01 16:54:00,730 - DEBUG - Contract call arguments: [[(erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp, 87908537530726782098814095450)]]
2025-06-01 16:54:00,747 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/3898e2fe578c852878712fba445f0b1fb93451db888c4c09e4d2742636856baa


'3898e2fe578c852878712fba445f0b1fb93451db888c4c09e4d2742636856baa'

In [26]:
advance_blocks(5)

In [33]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher


data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
week = data_fetcher.get_data("getCurrentWeek")
print(f"Current week: {week}")

Current week: 245


Incentivize farm

In [34]:
from multiversx_sdk.abi import Abi

from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher
from utils.utils_tx import ESDTToken

abi = Abi.load(PROJECT_ROOT / "wasm/mex-governance/mex-governance.abi.json")

incentivize_amount = 100 * 10**18  # 100 MEX

args: list[list[Any]] = []
args.append((WrapperAddress("erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp"), incentivize_amount, week+1))


# mex_governance_contract.address = WrapperAddress("erd1qqqqqqqqqqqqqpgq58wgq27e3l475wlcq570xkjc6cmehdlv2jpsgp684p")
tokens = [
    ESDTToken(energy_contract.base_token, 0, incentivize_amount)
]

sc_args = [
    tokens,
    args
    ]
mex_governance_contract.incentivize_farm(context.deployer_account, proxy, abi, sc_args)

2025-06-02 11:21:33,965 - INFO - Set incentive farm
2025-06-02 11:21:33,965 - DEBUG - Set incentive farm
2025-06-02 11:21:33,969 - DEBUG - Contract call arguments: [[(erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp, 100000000000000000000, 246)]]
2025-06-02 11:21:33,973 - INFO - No explorer known for http://localhost:8085. transaction raw path: http://localhost:8085/transaction/0db47539c81168b09fdff51bc1d195f9bcf5ff968e2cb2b390eb72c0f35ad48e


'0db47539c81168b09fdff51bc1d195f9bcf5ff968e2cb2b390eb72c0f35ad48e'

In [35]:
advance_blocks(5)

In [ ]:
context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")


In [ ]:
context.deployer_account.sync_nonce(proxy)
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, farm)

Claim incentive

In [ ]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher


data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
week = data_fetcher.get_data("getCurrentWeek")
print(f"Current week: {week}")
mex_governance_contract.claim_incentive(context.deployer_account, proxy, [week])

SET FARM EMISSION

In [ ]:
mex_governance_contract.set_farm_emissions(context.deployer_account, proxy, [])

Blacklist Farm

In [ ]:
mex_governance_contract.blacklist_farm(deployer_account=context.deployer_account, network_provider=context.network_provider.proxy)

FUND ACCOUNTS

In [15]:
import json
with open(PROJECT_ROOT / "states" / "0_erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
from utils.utils_chain import Account, WrapperAddress, nominated_amount
from utils.utils_tx import endpoint_call

sender = Account(pem_file=config.DEFAULT_ACCOUNTS)
sender.address = WrapperAddress("erd1rf4hv70arudgzus0ymnnsnc4pml0jkywg2xjvzslg0mz4nn2tg7q7k0t6p")
receiver = WrapperAddress("erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
amount = nominated_amount(200)

sender.sync_nonce(proxy)
endpoint_call(proxy, 100000, sender, receiver, "cado", [], amount)

advance_epoch(1)

GET DATA

In [ ]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher

week = 1
token_identifier = "MEX-455c57"
data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
all_week_emissions = data_fetcher.get_data("getAllWeekEmissions", [week])
print(all_week_emissions)

In [ ]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher

data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
all_week_emissions = data_fetcher.get_data("getCurrentWeek", [])
print(all_week_emissions)

In [ ]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher
from utils.utils_chain import base64_to_hex, decode_merged_attributes

data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
all_week_emissions = data_fetcher.get_data("getFirstWeekStartEpoch", [])
print(all_week_emissions)

In [ ]:
from utils.contract_data_fetchers import MEXGovernanceContractDataFetcher
from utils.utils_chain import hex_to_base64, hex_to_string

data_fetcher = MEXGovernanceContractDataFetcher(WrapperAddress(mex_governance_contract.address), context.network_provider.proxy.url)
energy_factory_address = data_fetcher.get_data("getEnergyFactoryAddress", [])
print(f"Energy factory address: {energy_factory_address}")


In [24]:
energy_contract.get_energy_for_user(proxy, users[1].address.to_bech32())

{'amount': 87908537530726782098814095450,
 'last_update_epoch': 1708,
 'total_locked_tokens': 66490095120516455474008930}

GET USER ENERGY

In [ ]:
from utils.decoding_structures import ENERGY_ENTRY
from utils.utils_chain import string_to_hex


config.DEFAULT_WORKSPACE
with open(config.DEFAULT_WORKSPACE / "deploy" / "configs-mainnet" / "upgrader_outputs" / "energy-contract-state.json", "r") as f:
    energy_contract_state = json.load(f)

users = []
for key, value in energy_contract_state.items():
    if string_to_hex("userEnergy") not in key:
        continue
    user_address = WrapperAddress.from_hex(key.replace(string_to_hex("userEnergy"), ""))
    user_energy_entry = decode_merged_attributes(value, ENERGY_ENTRY)

    user_dict = {
        "address": user_address.bech32(),
        "energy": user_energy_entry.get("amount"),
        "last_update_epoch": user_energy_entry.get("last_update_epoch"),
        "total_locked_tokens": user_energy_entry.get("total_locked_tokens"),
        "tokens": [],
        "metabonding_tokens": 0
    }
    users.append(user_dict)

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "all_users_energy.json", "w") as f:
    json.dump(users, f, indent=4)

print(f"Done. Found {len(users)} users")
